# 01 - Ingesta y Limpieza de Datos

**Proyecto:** Seguridad Inteligente con Arduino y Visión Artificial

Este notebook realiza la ingesta de datos desde la exportación de Telegram y ejecuta el proceso de limpieza inicial.

---

## 1. Configuración Inicial

In [ ]:
# Importar librerías necesarias
import sys
import os

# Importaciones estándar
import pandas as pd
import numpy as np
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Importar módulo personalizado (ahora definido en la siguiente celda)
# from ingest import TelegramDataIngestor

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Opciones de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Librerías importadas correctamente")

In [ ]:
# Clase TelegramDataIngestor (sustituye al módulo src/ingest.py)
import yaml
from datetime import datetime
from typing import Dict, List, Optional

class TelegramDataIngestor:
    """Clase para ingerir y procesar datos de exportación de Telegram."""
    
    def __init__(self, config_path: str = "configs/config.yaml"):
        """
        Inicializa el ingestor de datos.
        
        Args:
            config_path: Ruta al archivo de configuración YAML
        """
        with open(config_path, 'r') as f:
            self.config = yaml.safe_load(f)
        
        self.raw_path = Path(self.config['data']['raw_path'])
        self.processed_path = Path(self.config['data']['processed_path'])
        self.result_file = self.config['telegram']['result_file']
        
    def load_telegram_export(self) -> Dict:
        """Carga el archivo result.json de la exportación de Telegram."""
        file_path = self.raw_path / self.result_file
        if not file_path.exists():
            raise FileNotFoundError(f"No se encontró el archivo: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✓ Archivo cargado: {file_path}")
        print(f"  Total de mensajes: {len(data.get('messages', []))}")
        return data

    def _extract_text(self, text) -> str:
        """Extrae el texto de diferentes formatos de mensaje."""
        if isinstance(text, str):
            return text
        elif isinstance(text, list):
            return ' '.join([t if isinstance(t, str) else t.get('text', '') for t in text])
        return ''

    def extract_messages(self, data: Dict) -> pd.DataFrame:
        """Extrae y procesa los mensajes del export de Telegram."""
        messages = data.get('messages', [])
        records = []
        for msg in messages:
            record = {
                'id': msg.get('id'),
                'type': msg.get('type'),
                'date': msg.get('date'),
                'from': msg.get('from'),
                'from_id': msg.get('from_id'),
                'text': self._extract_text(msg.get('text', '')),
                'photo': msg.get('photo'),
                'has_photo': bool(msg.get('photo')),
                'media_type': msg.get('media_type'),
            }
            records.append(record)
        df = pd.DataFrame(records)
        print(f"✓ Extraídos {len(df)} registros")
        return df

    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Limpia y procesa el DataFrame de mensajes."""
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['hour'] = df['date'].dt.hour
        df['day_of_week'] = df['date'].dt.dayofweek
        df['day_name'] = df['date'].dt.day_name()
        df = df.drop_duplicates(subset=['id'])
        df = df.sort_values('date').reset_index(drop=True)
        print(f"✓ Datos limpios: {len(df)} registros únicos")
        print(f"  Rango de fechas: {df['date'].min()} a {df['date'].max()}")
        return df

    def save_processed_data(self, df: pd.DataFrame, filename: Optional[str] = None):
        """Guarda los datos procesados en formato CSV."""
        if filename is None:
            filename = self.config['data']['output_file']
        output_path = self.processed_path / filename
        output_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✓ Datos guardados en: {output_path}")

    def run(self):
        print("=== Iniciando ingesta de datos ===\n")
        data = self.load_telegram_export()
        df = self.extract_messages(data)
        df_clean = self.clean_data(df)
        self.save_processed_data(df_clean)
        print("\n=== Ingesta completada ===")
        return df_clean

## 2. Ingesta de Datos

Cargaremos los datos desde el archivo `result.json` exportado de Telegram.

In [ ]:
# Inicializar el ingestor
ingestor = TelegramDataIngestor(config_path='../configs/config.yaml')

print("✓ Ingestor inicializado")
print(f"  Ruta de datos raw: {ingestor.raw_path}")
print(f"  Ruta de datos procesados: {ingestor.processed_path}")

In [ ]:
# Cargar archivo de exportación de Telegram
try:
    data = ingestor.load_telegram_export()
    print("\n✓ Datos cargados exitosamente")
    print(f"  Claves disponibles: {list(data.keys())}")
except FileNotFoundError as e:
    print(f"⚠ Error: {e}")
    print("\nPor favor, coloca el archivo 'result.json' en: data/raw/telegram_export/")
    data = None

## 3. Extracción y Procesamiento de Mensajes

In [ ]:
# Extraer mensajes del archivo JSON
if data is not None:
    df_raw = ingestor.extract_messages(data)
    
    print("\n=== Vista previa de los datos crudos ===")
    display(df_raw.head())
    
    print("\n=== Información del DataFrame ===")
    print(df_raw.info())
else:
    print("⚠ No se pueden extraer mensajes sin datos")

## 4. Limpieza de Datos

In [ ]:
# Limpiar y procesar los datos
if data is not None:
    df_clean = ingestor.clean_data(df_raw)
    
    print("\n=== Datos limpios ===")
    display(df_clean.head())
    
    print("\n=== Estadísticas básicas ===")
    print(f"Total de registros: {len(df_clean)}")
    print(f"Rango de fechas: {df_clean['date'].min()} a {df_clean['date'].max()}")
    print(f"Columnas: {list(df_clean.columns)}")
else:
    print("⚠ No se pueden limpiar datos sin información previa")

## 5. Exploración Rápida

In [ ]:
# Verificar valores nulos
if 'df_clean' in locals():
    print("=== Valores nulos por columna ===")
    null_counts = df_clean.isnull().sum()
    print(null_counts[null_counts > 0])
    
    if null_counts.sum() == 0:
        print("✓ No hay valores nulos")

In [ ]:
# Distribución de tipos de mensajes
if 'df_clean' in locals() and 'type' in df_clean.columns:
    print("=== Distribución de tipos de mensajes ===")
    print(df_clean['type'].value_counts())
    
    # Visualización
    plt.figure(figsize=(10, 5))
    df_clean['type'].value_counts().plot(kind='bar', color='steelblue')
    plt.title('Distribución de Tipos de Mensajes')
    plt.xlabel('Tipo')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Eventos con fotos
if 'df_clean' in locals() and 'has_photo' in df_clean.columns:
    print("=== Eventos con fotografías ===")
    photo_counts = df_clean['has_photo'].value_counts()
    print(photo_counts)
    print(f"\nPorcentaje con fotos: {(photo_counts.get(True, 0) / len(df_clean) * 100):.2f}%")

## 6. Guardar Datos Procesados

In [ ]:
# Guardar el DataFrame limpio
if 'df_clean' in locals():
    ingestor.save_processed_data(df_clean)
    print("\n✓ Datos guardados exitosamente")
    print(f"  Archivo: {ingestor.processed_path / ingestor.config['data']['output_file']}")
else:
    print("⚠ No hay datos para guardar")

## 7. Resumen

In [ ]:
# Resumen final
if 'df_clean' in locals():
    print("=" * 60)
    print("RESUMEN DE INGESTA Y LIMPIEZA")
    print("=" * 60)
    print(f"✓ Registros totales: {len(df_clean)}")
    print(f"✓ Columnas: {len(df_clean.columns)}")
    print(f"✓ Rango temporal: {(df_clean['date'].max() - df_clean['date'].min()).days} días")
    print(f"✓ Datos guardados en: data/processed/events_clean.csv")
    print("\n✓ Proceso completado exitosamente")
    print("\nPróximo paso: Ejecutar notebook 02_eda_y_features.ipynb")
    print("=" * 60)
else:
    print("⚠ El proceso no se completó. Verifica que los datos estén disponibles.")